**Install Packages**

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

#app = IPython.Application.instance()
#app.kernel.do_shutdown(True)

**WAIT**

In [3]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [4]:
# Define project information
PROJECT_ID = "regbrain-poc"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part

In [6]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

In [8]:
import http.client
import typing
import urllib.request
import os

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


#def get_url_from_gcs(gcs_uri: str) -> str:
#    # converts gcs uri to url for image display.
#    url = "https://storage.googleapis.com/" + gcs_uri.replace("gs://", "").replace(
#        " ", "%20"
#    )
#    return url


def print_multimodal_prompt(contents: list):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if isinstance(content, Image):
            display_images([content])
        elif isinstance(content, Part):
            url = get_url_from_gcs(content.file_data.file_uri)
            IPython.display.display(load_image_from_url(url))
        else:
            print(content)

In [11]:
import pandas as pd
from vertexai import generative_models

safety_config = {
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_UNSPECIFIED: generative_models.HarmBlockThreshold.BLOCK_NONE,
}

# Generation config
gen_config = generative_models.GenerationConfig(
    temperature=0.0,
    candidate_count=1,
)

In [10]:
url = "https://www.fema.gov/print/pdf/node/664453"

In [42]:
import pandas as pd
from vertexai import generative_models

safety_config = {
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_UNSPECIFIED: generative_models.HarmBlockThreshold.BLOCK_NONE,
}

# Local Image
#! gsutil cp "/content/sample_data/Screenshot 2024-02-16 085333.png" ./image.jpg
folder = "/content/drive/MyDrive/Images"

csv_file_path = '/content/drive/MyDrive/first100.csv'
df = pd.read_csv(csv_file_path)
df['gemini_second'] = ''
i = 0

# Generation config
gen_config = generative_models.GenerationConfig(
    temperature=0.0,
    candidate_count=1,
)

for img in os.listdir(folder):
  i += 1
  # print('Document No.', i, img)
  # Load from local file
  image = Image.load_from_file(folder + "/" + img)
"""You are tasked with extracting the highest quality title from this document.
  The purpose of the title is to be displayed within a list of other document titles.
  The title should give a person looking at the list the clearest possible idea of what the document is about.
  The title extracted should be document specific, usually taking the form of a descriptive sentence.
  The preferred title is one that gives the user a clear description of the content or intended purpose of the document.
  MUST be distinct from the body text either in style or position. Body text shouldn't appear before the extracted title.
  MUST distinguish this document from others by the same publisher or in the same series of documents.
  MUST NOT be a subheading that only refers to a part of the document.
  MUST include a unique identifier such as a citation of reference too, if available.
  MUST NOT be the publishers name or address unless they are specific to this document, or unless they are included in an otherwise suitable title sentence.
  MUST include subtitles where possible.
  If no adequate descriptive title can be found taking a document name or citation is the next choice of title to extract.
  Any outputted text must appear verbatim within the document.
  Text case should be preserved.
  Please follow these guidelines in order to extract a title from this image.
  If you can't identify a title that fits the guidelines, please return 'No title found.'"""
  # Prepare contents
  prompt = """You are tasked with extracting the highest quality title from this document.
  The purpose of the title is to be displayed within a list of other document titles.
  The title should give a person looking at the list the clearest possible idea of what the document is about.
  The title extracted should be document specific, usually taking the form of a descriptive sentence.
  The preferred title is one that gives the user a clear description of the content or intended purpose of the document.
  MUST be distinct from the body text either in style or position. Body text shouldn't appear before the extracted title.
  MUST distinguish this document from others by the same publisher or in the same series of documents.
  MUST NOT be a subheading that only refers to a part of the document.
  MUST include a unique identifier such as a citation of reference too, if available.
  MUST NOT be the publishers name or address unless they are specific to this document, or unless they are included in an otherwise suitable title sentence.
  MUST include subtitles where possible.
  If no adequate descriptive title can be found taking a document name or citation is the next choice of title to extract.
  Any outputted text must appear verbatim within the document.
  Text case should be preserved.
  Please follow these guidelines in order to extract a title from this image.
  If you can't identify a title that fits the guidelines, please return 'No title found.'"""
  contents = [image, prompt]

  response = multimodal_model.generate_content(contents, safety_settings=safety_config, generation_config=gen_config)
  try:
      df.loc[df['file_upload'] == img, 'gemini_second'] = response.candidates[0].text
      print(response.candidates[0].text)
  except AttributeError as e:
    print("An error occured: ", e)

df.to_csv(csv_file_path, index=False)

 Definition of Abortion; Medication Abortion Reversal Notification; HB 2264
 Temporary Joint Rules of the Senate and House of Representatives 93rd Legislature (2023-2024)
 Equal Employment Opportunity Data Posted Pursuant to the No Fear Act: HUD (and below)
 Prevention, management and better care of diabetes in the EU on the occasion of World Diabetes Day
 TRS' Report to Legislative Fiscal Division Pursuant to 2-6-1033, MCA, for Quarter Ending September 30, 2023
 COUNTERPARTY CREDIT RISK MONITORING POLICY
 Questions and Answers
On MiFID II and MiFIR investor protection and intermediaries topics
 Explanatory Statement
Airspace Regulations 2007
Civil Aviation Safety Regulations 1998
CASA OAR 048/23 – Declarations and Directions in relation to Prohibited,
Restricted and Danger Areas etc. - Permanent Instrument 2023
 Federal Election Commission Legislative Recommendations 2023
 PET INSURANCE MODEL ACT
 House Calendar, Thursday, April 20, 2023
 SBA Information Notice: Issuance of SOP 50 10 

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x787acc38e390>